# Wind Plants

# Data Import

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from folium import plugins
import folium


pd.set_option('display.float_format', lambda x: '%0.4f' % x)

/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## US Power Plants Data

In [2]:
gisfilepath = "/Users/jnapolitano/Projects/gas.jnapolitano.io/source/data/Power_Plants.geojson"

powerplants_df = gpd.read_file(gisfilepath)
#Selecting only Operational Plants
powerplants_df=powerplants_df.loc[powerplants_df['STATUS'] == 'OP'].copy()
na = powerplants_df.COAL_USED.min()
powerplants_df.replace(na, 0 , inplace=True)

#powerplants_df = powerplants_df.to_crs(epsg=3857)

powerplants_df.describe()


,OBJECTID,LATITUDE,LONGITUDE,OPER_CAP,SUMMER_CAP,WINTER_CAP,PLAN_CAP,RETIRE_CAP,GEN_UNITS,PLAN_UNIT,RETIR_UNIT,COAL_USED,NGAS_USED,OIL_USED,NET_GEN,CAP_FACTOR,LINES,SOURCE_LAT,SOURC_LONG
count,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000,8736.0000
mean,5709.4960,39.0405,-95.0667,134.5691,123.2273,127.9356,1.5068,12.3725,2.3368,0.0144,0.2098,75285.8993,1351867.2459,3590.2573,466814.5072,0.0371,0.3489,39.0424,-94.1651
std,3315.5893,5.5191,19.0667,367.0281,339.7523,350.9757,36.1032,100.9833,2.8505,0.1797,0.9739,555638.6353,6289781.4216,57026.9476,1813781.4037,12.5042,0.5880,5.5178,21.7038
min,1.0000,18.9737,-171.7124,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-3465212.0000,-1141.5514,0.0000,18.9742,-171.7124
25%,2666.7500,35.1990,-113.7045,2.7000,2.5000,2.4000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2141.7500,0.0574,0.0000,35.1986,-113.6161
50%,5832.5000,39.4456,-91.6117,9.6000,8.7500,8.8000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,11727.0000,0.2137,0.0000,39.4492,-91.0609
75%,8566.2500,42.4995,-78.6045,74.8000,70.0000,72.0000,0.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,133402.5000,0.4291,1.0000,42.5003,-78.2997
max,11810.0000,71.2921,-67.4016,6809.0000,7079.0000,7079.0000,2200.0000,3623.0000,73.0000,5.0000,28.0000,10040760.0000,138014506.0000,4130642.0000,32377477.0000,4.6214,11.0000,71.2920,36.2439


## Identifying the Wind Plants in the Data Set

In [3]:
#powerplants_df.PRIM_FUEL.unique()
#RC,WC,NUC
#Selecting only Operational Plants
#powerplants_df=powerplants_df.loc[powerplants_df['STATUS'] == 'OP'].copy()

#selecting coal plants
wind_plants = powerplants_df.loc[powerplants_df['PRIM_FUEL'] == 'WND'].copy()

#concatenating coal type dfs into one
wind_plants.describe()




,OBJECTID,LATITUDE,LONGITUDE,OPER_CAP,SUMMER_CAP,WINTER_CAP,PLAN_CAP,RETIRE_CAP,GEN_UNITS,PLAN_UNIT,RETIR_UNIT,COAL_USED,NGAS_USED,OIL_USED,NET_GEN,CAP_FACTOR,LINES,SOURCE_LAT,SOURC_LONG
count,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000,1043.0000
mean,5708.0221,40.2198,-99.0458,82.4783,81.9475,82.0477,0.4411,0.8269,1.1678,0.0115,0.0201,0.0000,0.0000,0.0000,219515.7784,-0.8512,0.2991,40.2285,-96.0238
std,3546.5394,5.0681,14.3208,88.5169,88.4435,88.3916,12.3696,11.4182,0.5907,0.1153,0.1768,0.0000,0.0000,0.0000,270760.0809,35.3758,0.4725,5.0641,24.3758
min,76.0000,18.9737,-158.0408,0.1000,0.1000,0.1000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-1141.5514,0.0000,18.9742,-158.0408
25%,1877.5000,36.2779,-105.1253,9.9000,9.8000,9.8000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,11530.5000,0.2355,0.0000,36.2753,-105.5026
50%,7667.0000,41.4700,-98.0297,56.7000,53.0000,55.2000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,119888.0000,0.3027,0.0000,41.4547,-97.6736
75%,8342.0000,43.7654,-93.2250,133.5000,132.0000,132.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,344625.5000,0.3623,1.0000,43.7578,-92.4863
max,11808.0000,64.0583,-67.8105,735.5000,735.5000,735.5000,398.8000,217.5000,9.0000,2.0000,3.0000,0.0000,0.0000,0.0000,2159583.0000,2.4250,2.0000,64.0583,36.2439


## Wind Plants Interactive Map

```{eval-rst}

.. index::
   single: Wind Plants Interactive Map

```

In [4]:
wind_map = wind_plants.explore(column="NET_GEN", # make choropleth based on "BoroName" column
     popup=True, # show all values in popup (on click)
     tiles="Stamen Terrain", # use "CartoDB positron" tiles
     cmap="Set1", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black"),
     marker_kwds= dict(radius=6),# use black outline)
     scheme = 'EqualInterval',
     k = 5

)
wind_map

## Wind Plant Density Interactive Map

```{eval-rst}

.. index::
   single: Wind Plant Density Interactive Map

```

In [5]:
wind_heat_data = [[point.xy[1][0], point.xy[0][0]] for point in wind_plants.geometry ]
#

wind_density_map = folium.Map(location = [30, -90], tiles='Cartodb dark_matter', zoom_start = 4)

wind_heat_layer = plugins.HeatMap(data = wind_heat_data)
wind_heat_layer.add_to(wind_density_map)

wind_density_map

